# Deal or No Deal
*A Monte Carlo Simulation Example*

In [1]:
%pip install fsspec --quiet
%pip install s3fs --quiet
%pip install pyspark --quiet


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark.sql.functions as F
import random
import math
from config import Config
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

experiment_config = Config()

24/08/02 18:03:23 WARN Utils: Your hostname, codespaces-7ca8c5 resolves to a loopback address: 127.0.0.1; using 10.0.1.245 instead (on interface eth0)
24/08/02 18:03:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/02 18:03:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Perform Monte Carlo loop

for i_trial in range(1, experiment_config.n_trials+1):
    print(f'TRIAL #{i_trial}:')

    # Initialize trial variables
    l_case_values = experiment_config.l_all_case_values.copy()
    l_highest_offer = [i_trial, 0, 0] # [i_trial, i_round, value_offer]

    # Play the game 
    for i_round in experiment_config.dict_cases_per_round.keys():
        n_picks = experiment_config.dict_cases_per_round[i_round]
        print(f'\n It is round #{i_round}, pick {n_picks} case(s).')
        for i_case in range(1, n_picks+1):
            print(f' Case {i_case}')
            value_case = random.choice(l_case_values)
            l_case_values.remove(value_case)
            n_cases_remaining = len(l_case_values)
        print(f' Cases remaining after selection: {n_cases_remaining}')
        l_case_values_squared = [i ** 2 for i in l_case_values]
        value_offer = round(math.sqrt(sum(l_case_values_squared) / n_cases_remaining),2)
        print(f' Banker offer: ${value_offer}')
        l_offer = [i_trial, i_round, value_offer]
        if value_offer > l_highest_offer[2]:
            l_highest_offer = l_offer
        experiment_config.l_trial_results.append(l_offer)

    
    experiment_config.l_trial_highest_results.append(l_highest_offer)

# Print experiment results   
print(experiment_config.l_trial_highest_results)

TRIAL #1:

 It is round #0, pick 1 case(s).
 Case 1
 Cases remaining after selection: 25
 Banker offer: $288539.57

 It is round #1, pick 6 case(s).
 Case 1
 Case 2
 Case 3
 Case 4
 Case 5
 Case 6
 Cases remaining after selection: 19
 Banker offer: $150241.36

 It is round #2, pick 5 case(s).
 Case 1
 Case 2
 Case 3
 Case 4
 Case 5
 Cases remaining after selection: 14
 Banker offer: $112210.54

 It is round #3, pick 4 case(s).
 Case 1
 Case 2
 Case 3
 Case 4
 Cases remaining after selection: 10
 Banker offer: $132768.86

 It is round #4, pick 3 case(s).
 Case 1
 Case 2
 Case 3
 Cases remaining after selection: 7
 Banker offer: $48219.1

 It is round #5, pick 2 case(s).
 Case 1
 Case 2
 Cases remaining after selection: 5
 Banker offer: $57009.76

 It is round #6, pick 1 case(s).
 Case 1
 Cases remaining after selection: 4
 Banker offer: $63738.85

 It is round #7, pick 1 case(s).
 Case 1
 Cases remaining after selection: 3
 Banker offer: $45645.6

 It is round #8, pick 1 case(s).
 Case 

24/08/02 18:03:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
# Store results to Spark DataFrame

sdf_trial_results = spark.createDataFrame(experiment_config.l_trial_results).toDF('trial', 'round', 'offer')
sdf_trial_results.show(10)

sdf_trial_highest_results = spark.createDataFrame(experiment_config.l_trial_results).toDF('trial', 'highest_round', 'offer')
sdf_trial_highest_results.show(10)

# Write results to parquet

sdf_trial_results.write.parquet(path = "trial_results.parquet", mode="overwrite")
sdf_trial_highest_results.write.parquet(path = "sdf_trial_highest_results.parquet", mode="overwrite")

+-----+-----+---------+
|trial|round|    offer|
+-----+-----+---------+
|    1|    0|288539.57|
|    1|    1|150241.36|
|    1|    2|112210.54|
|    1|    3|132768.86|
|    1|    4|  48219.1|
|    1|    5| 57009.76|
|    1|    6| 63738.85|
|    1|    7|  45645.6|
|    1|    8| 53035.66|
|    1|    9|  75000.0|
+-----+-----+---------+
only showing top 10 rows

+-----+-------------+---------+
|trial|highest_round|    offer|
+-----+-------------+---------+
|    1|            0|288539.57|
|    1|            1|150241.36|
|    1|            2|112210.54|
|    1|            3|132768.86|
|    1|            4|  48219.1|
|    1|            5| 57009.76|
|    1|            6| 63738.85|
|    1|            7|  45645.6|
|    1|            8| 53035.66|
|    1|            9|  75000.0|
+-----+-------------+---------+
only showing top 10 rows

